In [1]:
import boto3
import os

In [3]:
access_key_id = "minioadmin"
secret_access_key = "minioadmin"
minio_url = "http://localhost:9000"

minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

In [6]:
from PIL import Image
import io
from tqdm import tqdm

bucket_origen = "persistent-landing"
bucket_desti = "formatted-zone"
prefix_origen = "imagenes/"
mida_final = (600, 450)

paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_origen, Prefix=prefix_origen):
    for obj in tqdm(page.get("Contents", []), desc="Processant imatges"):
        key = obj["Key"]
        filename = key.split("/")[-1]

        if not filename.lower().endswith(".png"):
            continue
        #Llegir la imatge des del bucket
        response = minio_client.get_object(Bucket=bucket_origen, Key=key)
        image_data = response["Body"].read()
        #Obrir amb PIL per validar i redimensionar
        try:
            img = Image.open(io.BytesIO(image_data)).convert("RGB")
        except Exception as e:
            print(f"Error amb {filename}: {e}")
            continue

        #Redimensionar a mida consistent
        img = img.resize(mida_final)

        #Desa a memòria i puja a formatted-zone
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        buffer.seek(0)

        new_key = f"imagenes/{filename}"
        minio_client.put_object(
            Bucket=bucket_desti,
            Key=new_key,
            Body=buffer
        )

Processant imatges: 100%|██████████| 12/12 [00:00<00:00, 28.43it/s]
